# PMR3508 - Tarefa 1 - Segunda etapa: análise da base HouseholdIncome
#baseline: 0.220
#melhor tentativa: corr >0.2, score 0.309
Nesta etapa, será realizado o mesmo procedimento da primeira, mas para a base Costa Rican Household.

## Parte 1: Data Cleaning

Visualizando dados faltantes, e como tratar o dataset de modo a manter o máximo da informação original de um modo que possa ser aproveitado pelo modelo preditivo.

In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [ ]:
raw_train_base = pd.read_csv("../input/train.csv")
raw_train_base.info()
clean = raw_train_base

In [ ]:
null_columns = raw_train_base.columns[raw_train_base.isnull().any()]
raw_train_base[null_columns].isnull().sum()

In [ ]:
#removendo atributos com muitos dados faltatantes
clean = clean.drop(columns=['v2a1','v18q1','rez_esc'])
clean = clean.dropna()

## Segunda etapa: EDA

Explorando o dataset...

In [ ]:
clean.head()

In [ ]:
clean.loc[:,'tamhog'].value_counts().plot(kind='bar')

In [ ]:
clean.loc[:,'escolari'].value_counts().plot(kind='bar')

In [ ]:
clean.loc[:,'bedrooms'].value_counts().plot(kind='bar')

In [ ]:
clean.loc[:,'overcrowding'].value_counts().plot(kind='bar')

In [ ]:
clean.loc[:,'Target'].value_counts().plot(kind='bar')

In [ ]:
plt.matshow(clean.corr())

In [ ]:
anl0 = clean.corr().loc[:,'Target'].sort_values(ascending=True)
anl0

In [ ]:
clean = clean.drop(columns = 'elimbasu5')

In [ ]:
clean.select_dtypes(include= 'object').head()

In [ ]:
clean.loc[:,'idhogar'].nunique()

## Terceira parte: Data preparation

Preparando os dados para o modelo preditivo.

In [ ]:
train_clean = clean.replace(to_replace=['yes','no'], value = [1,0])
train_clean = train_clean.drop(columns='idhogar')
train_clean.loc[:,['edjefe','edjefa','dependency']] = train_clean.loc[:,['edjefe','edjefa','dependency']].astype('float64')

In [ ]:
index = anl0.where(lambda x : abs(x) > 0.2).dropna().index[1:-1]

In [ ]:
X_train = train_clean.loc[:,index]
Y_train = train_clean.iloc[:,-1]

In [ ]:
raw_test_base = pd.read_csv('../input/test.csv')
test_clean = raw_test_base.drop(columns=['v2a1','v18q1','rez_esc','elimbasu5'])
test_clean = test_clean.replace(to_replace=['yes','no'], value = [1,0])
test_clean = test_clean.drop(columns='idhogar')
test_clean.loc[:,['edjefe','edjefa','dependency']] = test_clean.loc[:,['edjefe','edjefa','dependency']].astype('float64')

In [ ]:
test_clean = test_clean.fillna(0.0)

In [ ]:
X_test = test_clean.loc[:,index]
X_test.info()
X_train.info()

## Quarta Parte: Modelo preditivo: kNN com GridSearchCV

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
#usarei gridsearch para verificar os melhores hiperparâmetros do kNN.
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']
p_options = list(range(1,3))
param_grid = dict(n_neighbors=k_range, p=p_options)#, p=p_options

In [ ]:
#inicializando o classificador a ser verificado no GridSearchCV
knn = KNeighborsClassifier(n_neighbors=5)

from sklearn.model_selection import cross_val_score
#inicializando o GridSearchCV
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', n_jobs = -2)  

In [ ]:
grid.fit(X_train, Y_train)
print(grid.best_estimator_)
print(grid.best_score_)

In [ ]:
f_kNN = grid.best_estimator_

In [ ]:
f_kNN.fit(X_train,Y_train)

In [ ]:
Y_test = f_kNN.predict(X_test)

In [ ]:
output = pd.DataFrame({'Id':test_clean.loc[:,'Id'],'Target':Y_test})

In [ ]:
output.to_csv("submitHouseholdIncome5.csv", index = False)